В прошлом отчете мы провели кластеризацию пользователей. Алгоритм DBSCAN, на мой взгляд, отлично справился с этой задачей и разделил пользователей на 2 большие группы. Попробуем реализовать классификатор, используя полученные метки из кластеризации

In [10]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn import decomposition
from sklearn import metrics
from sklearn.cluster import DBSCAN

from pickle import dump

sns.set(rc={'figure.figsize':(8,6)}, palette="muted")
# sns.set_style("whitegrid")

df = pd.read_table('./data/test_data.tsv')

df.head()

,eid,t,user,tn,geo,cnethash,devtype,brotype,bsize,siteid,domhash,urlhash
0,347854231724,1050848,21,0,3818162082,8335634595980108828,2,2,1642713892,1732823002,17261330804588317013,16521739535555380904
1,329455126064,694606,36,0,3818162082,2563013194488493467,3,2,2286540757,673473971,8781226038565509786,14273372146021722044
2,329455254926,694659,36,0,3818162082,2563013194488493467,3,2,2286540757,673473971,8781226038565509786,14273372146021722044
3,329455273798,694596,36,0,3818162082,2563013194488493467,3,2,3529439892,673473971,8781226038565509786,14273372146021722044
4,329455293738,694608,36,0,3818162082,2563013194488493467,3,2,390304810,673473971,8781226038565509786,14273372146021722044


In [11]:
def tukey_noise(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    df = df[(df[column] >= Q1 - 1.5*IQR) & (df[column] <= Q3 + 1.5*IQR)]
    return df
# Считаем время, требущееся на клик
df_sort = df.sort_values(by=['user', 't'], ascending=True, axis=0)
df_sort['duration'] = df_sort.groupby(by=['user'])['t'].rolling(window=2).apply(lambda x: x.iloc[1] - x.iloc[0]).values
df_sort['flg'] = df_sort.groupby(by=['user'])['tn'].rolling(window=2).apply(lambda x: 1 if x.iloc[1] == 1 and x.iloc[0] == 0 else 0).values
df_sort.fillna(value=0, inplace=True)

# Убираем записи о показе рекламы. Оставляем только клики
df_user_click = df_sort[df_sort['flg'] == 1]
mean_val = df_user_click['duration'].mean()
median_val = df_user_click['duration'].median()

print(f'Среднее время, требующееся на клик: {mean_val}\nМедианное время, требующееся на клик: {median_val}')

n_init = df_user_click.shape[0]

# Логарифимруем время на клик, чтобы значения стали больше похожи на нормальное распределение
df_user_click['log_duration'] = np.log(df_user_click['duration'].values)
df_user_click = tukey_noise(df_user_click, 'log_duration')
n_temp = df_user_click.shape[0]
print(f'Удалено шумов {n_init - n_temp}')

# Выбираем столбцы, которые, по моему мнению, несут наиболее важную информацию и стандартизируем данные
df_cluster = df_user_click[['devtype', 'brotype', 'log_duration', 'domhash', 'siteid']]

scaler = StandardScaler()
minmax = MinMaxScaler()

data_scaled = scaler.fit_transform(df_cluster)
data_scaled = pd.DataFrame(data=data_scaled, columns=df_cluster.columns, index=df_cluster.index)

data_scaled = minmax.fit_transform(data_scaled)
data_scaled = pd.DataFrame(data=data_scaled, columns=df_cluster.columns, index=df_cluster.index)


pca = decomposition.PCA(n_components=2)
X_pca_ratio_2d = pd.DataFrame(pca.fit_transform(data_scaled), columns=(["col1", "col2"]))
# path_pca = './models/decomposition/pca.pkl'
# dump(pca, open(path_pca, 'wb'))

epsilon = 0.08
min_samples = 10

dbscan = DBSCAN(eps=epsilon, min_samples=min_samples)
dbscan.fit(X_pca_ratio_2d.values)

data_scaled['group'] = dbscan.labels_

Среднее время, требующееся на клик: 789.2948128693369
Медианное время, требующееся на клик: 7.0
Удалено шумов 80


<ipython-input-11-a1eb6cec13dc>:23: RuntimeWarning: divide by zero encountered in log
  df_user_click['log_duration'] = np.log(df_user_click['duration'].values)
<ipython-input-11-a1eb6cec13dc>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_user_click['log_duration'] = np.log(df_user_click['duration'].values)


Группы получились сбалансированные. Практически поровну значений как в первой, так и во второй группе. Флаг "-1"означает выброс в данных. Такую группу мы классифицировать не будем. Примением классификатор XGBoost.

In [12]:
data_scaled['group'].value_counts()

 1    768
 0    660
-1     15
Name: group, dtype: int64

In [13]:
from sklearn.model_selection import train_test_split

data_scaled = data_scaled[data_scaled['group'] != -1]
X, y =  data_scaled.iloc[:, :-1],  data_scaled.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [14]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report


model = xgb.XGBClassifier()


model.fit(X_train, y_train)

predictions = model.predict(X_test)

accuracy = accuracy_score(y_test, predictions)

print("Accuracy:", accuracy)
print("\nClassification Report:")
print(classification_report(y_test, predictions))

Accuracy: 1.0

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       190
           1       1.00      1.00      1.00       239

    accuracy                           1.00       429
   macro avg       1.00      1.00      1.00       429
weighted avg       1.00      1.00      1.00       429



Очень подозрительно, но классифкатор показал 100% точность. При этом подбором параметров модели я не занимался. Скорее всего либо ошибка в данных, либо в подходе к обучению. Рассмотрим классификатор "случайный лес".

In [15]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

y_test_predicted = clf.predict(X_test)
from sklearn.metrics import accuracy_score
print("Точность: {0:.2f}%".format(accuracy_score(y_test, y_test_predicted)*100))

Точность: 100.00%


Случайный лес также показал 100% точность. Как мне кажется, такой точностьи не может быть. Вопрос с обучением модели я выношу в доп. исследование.

Ответ на вопрос, поставленный в исследовании. В компании были накручены клики. Это видно на графиках, построенных в отчете по класстеризации. В группе 1, очень короткое время клика. Такое поведение присуще ботам, потому что они кликают на рекламу, как только увидят ее. Можно также провести доп исследование, возвращался ли пользователь после клика.